In [1]:
# import torch
# import torch.nn.functional as F
from huggingface_hub import login
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# import torch
# from transformers import GemmaTokenizerFast
# from transformers import Trainer, TrainingArguments
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# import os

In [2]:
!pip install --upgrade transformers


   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ------ --------------------------------- 1.6/10.4 MB 8.4 MB/s eta 0:00:02
   --------------- ------------------------ 3.9/10.4 MB 10.2 MB/s eta 0:00:01
   ----------------------- ---------------- 6.0/10.4 MB 9.7 MB/s eta 0:00:01
   --------------------------------- ------ 8.7/10.4 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 10.4/10.4 MB 10.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.29.3
    Uninstalling huggingface-hub-0.29.3:
      Successfully uninstalled huggingface-hub-0.29.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.0
    Uninstalling transformers-4.50.0:
      Successfully uninstalled transformers-4.50.0



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Load dataset
import pandas as pd
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Construct the full path to the dataset
data_path = os.path.join(project_dir, "data", "SVAMP.json")

# Load the dataset
data = pd.read_json(data_path)

# Display the first few rows of the dataset
data.head()



,ID,Body,Question,Equation,Answer,Type
0,chal-1,Each pack of dvds costs 76 dollars. If there i...,How much do you have to pay to buy each pack?,( 76.0 - 25.0 ),51,Subtraction
1,chal-2,Dan had $ 3 left with him after he bought a ca...,How much did the candy bar cost?,( 4.0 - 3.0 ),1,Subtraction
2,chal-3,Paco had 26 salty cookies and 17 sweet cookies...,How many salty cookies did Paco have left?,( 26.0 - 9.0 ),17,Subtraction
3,chal-4,43 children were riding on the bus. At the bus...,How many children got off the bus at the bus s...,( 43.0 - 21.0 ),22,Subtraction
4,chal-5,28 children were riding on the bus. At the bus...,How many more children got on the bus than tho...,( 30.0 - 28.0 ),2,Subtraction


In [4]:
login()

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm

# Step 3: Load the Gemma 2B Model and Tokenizer
model_name = "google/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

model-00001-of-00002.safetensors:  18%|#8        | 923M/4.99G [00:00<?, ?B/s]

C:\venvs\thesis\venv\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--google--gemma-2-2b-it. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.

In [4]:
# split for questions and answers
df = pd.DataFrame(data['Body'] + ', ' + data['Question'], columns=['text'])
df['label'] = data['Answer']

df.head()


,text,label
0,Each pack of dvds costs 76 dollars. If there i...,51
1,Dan had $ 3 left with him after he bought a ca...,1
2,Paco had 26 salty cookies and 17 sweet cookies...,17
3,43 children were riding on the bus. At the bus...,22
4,28 children were riding on the bus. At the bus...,2


In [25]:
from src.model_handlers.perturbator import Perturbator

perturbator = Perturbator(model, tokenizer)

In [26]:
results = []
for index, row in tqdm(df.iterrows(), total=len(df), desc="Evaluating"):
    input_text = row["text"]
    # Generate answers using the Perturbator
    generated_answers = perturbator._generate_samples(
        task="question-answering",
        input_text=input_text,
        num_samples=1,  # Generate one answer per input
        temperature=0.7  # Use default temperature
    )
    # Store results
    results.append({
        "input": input_text,
        "generated_answer": generated_answers[0],  # Take the first generated answer
        "expected_output": row["output"]
    })

Evaluating:   0%|          | 0/1000 [00:00<?, ?it/s]C:\venvs\thesis\venv\lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\venvs\thesis\venv\lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Evaluating:   0%|          | 0/1000 [00:00<?, ?it/s]


ValueError: Input length of input_ids is 184, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [ ]:
def evaluate_results(results):
    correct = 0
    for result in results:
        generated_answer = result["generated_answer"].strip().lower()
        expected_output = result["expected_output"].strip().lower()
        if generated_answer == expected_output:
            correct += 1
    acc = correct / len(results)
    return acc

accuracy = evaluate_results(results)
print(f"Model Accuracy on Test Set: {accuracy * 100:.2f}%")

# Step 8: Display Sample Results
for i, result in enumerate(results[:5]):  # Display first 5 results
    print(f"Example {i + 1}:")
    print(f"Input: {result['input']}")
    print(f"Generated Answer: {result['generated_answer']}")
    print(f"Expected Output: {result['expected_output']}")
    print("-" * 50)